In [1]:
from sumpy.recurrence import _make_sympy_vec, get_processed_and_shifted_recurrence

from sumpy.expansion.diff_op import (
    laplacian,
    make_identity_diff_op,
)

from sumpy.recurrence import get_recurrence, _generate_nd_derivative_relations, pde_to_ode_in_r, ode_in_r_to_x

import sympy as sp
from sympy import hankel1

import numpy as np

import math

import matplotlib.pyplot as plt
from matplotlib import cm, ticker

In [10]:
from sumpy.expansion.diff_op import DerivativeIdentifier, LinearPDESystemOperator
from immutabledict import immutabledict

In [22]:
def get_ode_in_x(x_order, y_order):
    single_partial = DerivativeIdentifier((x_order,y_order), 0)
    #Coefficients
    list_pde_dict = immutabledict({single_partial: 1})
    random_pde = LinearPDESystemOperator(2,(list_pde_dict,))

    ode_in_r_random, var, ode_order_random = pde_to_ode_in_r(random_pde)
    ode_in_x_random = ode_in_r_to_x(ode_in_r_random, var, ode_order_random).simplify()

    return ode_in_x_random

In [29]:
size = 3
table = []
for i in range(size):
    temp = []
    for j in range(size):
        temp.append(get_ode_in_x(i, j))
    table.append(temp)

In [30]:
table

[[f_x0, f_x1*x1/x0, (f_x1*x0**2 - f_x1*x1**2 + f_x2*x0*x1**2)/x0**3],
 [f_x1,
  x1*(-f_x1 + f_x2*x0)/x0**2,
  (-f_x1*x0**2 + 3*f_x1*x1**2 + f_x2*x0**3 - 3*f_x2*x0*x1**2 + f_x3*x0**2*x1**2)/x0**4],
 [f_x2,
  x1*(2*f_x1 - 2*f_x2*x0 + f_x3*x0**2)/x0**3,
  (2*f_x1*x0**2 - 12*f_x1*x1**2 - 2*f_x2*x0**3 + 12*f_x2*x0*x1**2 + f_x3*x0**4 - 5*f_x3*x0**2*x1**2 + f_x4*x0**3*x1**2)/x0**5]]